In [7]:
# !pip install selenium
# !pip install webdriver_manager
# !pip install clickhouse_driver
from selenium import webdriver
import time
from webdriver_manager.chrome import ChromeDriverManager
from clickhouse_driver import Client
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import os
from lxml import html
import re
import datetime

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
sec = 'FXES'
driver.get(f'https://www.tinkoff.ru/invest/stocks/{sec}/pulse/')



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/aleksandraromanenko/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


In [ ]:
page_length = driver.execute_script("return document.body.scrollHeight")
while page_length < 9898 + 1:
    try:
        driver.execute_script(f"window.scrollTo(0, {page_length - 1000});")
        page_length = driver.execute_script("return document.body.scrollHeight")
    except:
        pass

In [ ]:
source_data = driver.page_source
soup = bs(source_data, 'lxml')

In [ ]:
posts = soup.find_all('div', {'class':'PulsePostCollapsed__text_3nQ1n PulsePostCollapsed__fourOrFiveLines_qVFqa'})
logins = soup.find_all('div', {'class':'PulsePostAuthor__nicknameLink_1p4Zx'})
likes = soup.find_all('div', {'class':'PulsePostBody__likes_130ao'})
times = soup.find_all('div', {'class':'PulsePostAuthor__inserted_Ctbka'})
        
logins = [login.text for login in logins]
posts = [post.text for post in posts]
likes = [int(like.text.split()[0]) for like in likes]
times = [time.text for time in times]

df_posts = pd.DataFrame()
df_posts['login'] = logins
df_posts['post'] = posts
df_posts['likes'] = likes
df_posts['time'] = times


In [ ]:
def time(tim):
    if 'мин' in tim:
        ex = datetime.datetime.now() - datetime.timedelta(minutes=int(tim.split(' ')[0]))
        return ex.strftime(format='%d-%m-%Y %H:%m')
    elif 'час' in tim:
        if tim.split()[0] == 'час':
            ex = datetime.datetime.now() - datetime.timedelta(hours=1)
            return ex.strftime(format='%d-%m-%Y %H:%m')
        else:
            ex = datetime.datetime.now() - datetime.timedelta(hours=int(tim.split(' ')[0]))
            return ex.strftime(format='%d-%m-%Y %H:%m')
    elif 'вчера' in tim.lower():
        yester = datetime.datetime.now() - datetime.timedelta(days=1)
        return yester.strftime(format='%d-%m-%Y ') + tim.split(' ')[-1]

In [ ]:
df_posts.time = df_posts.time.apply(lambda x: time(x))

In [ ]:
df_posts['time'] = pd.to_datetime(df_posts['time'], format='%d-%m-%Y %H:%M')

In [ ]:
df_posts.head(3)

,login,post,likes,time
0,morcvec,"Посмотрел новый ETF от Финекса, $FXES, меня оч...",9,2021-07-15 15:07:00
1,BonSai_108,Добавляю в мой долгосрочный портфель (ИИС- дру...,4,2021-07-15 15:07:00
2,vova.bless,$FXES 🕹 FinEx запустил новый фонд на видеоигры...,1,2021-07-15 15:07:00


In [ ]:
from tqdm import tqdm 
logins = df_posts.login.unique()
subscribers = []
subscriptions = []
cap = [] 
deals = []
percents = []
colors = []
for login in tqdm(logins):
    url = f'https://www.tinkoff.ru/invest/social/profile/{login}/'
    response = requests.get(url)
    tree = bs(response.content, 'html.parser')
    subs = list(map(lambda x: x.split()[0], tree.find('div', {'class':'ProfileHeader__panel_3zzI4'}).text.split('•')))
    subscribers.append(subs[0])
    subscriptions.append(subs[1])
    
    summa = tree.find_all('div', {'class':'ProfileHeader__itemValue_2mH8M'})[0].text.replace('\xa0', ' ')
    amount = ''.join(summa.split( )[1:-1])
    cap.append(summa.split( )[0] + ' ' + amount + ' ' + summa.split( )[-1])
    
    deals.append(int(tree.find_all('div', {'class':'ProfileHeader__itemValue_2mH8M'})[1].text.split()[0]))
    
    
    percent = float(tree.find_all('div', {'class':'ProfileHeader__itemValue_2mH8M'})[2].text.split(
    )[0].split('%')[0])
    color = str(tree.find_all('span', {'class':'Icon-module__container_2fX7e'})[0]).split(
        'color:')[1].split(';')[0]
    
    if color == '#22a053':
        percents.append(percent)
    elif color == '#dd5656':
        percents.append(percent * -1)
    else:
        percents.append('-')
    
    

100%|██████████| 28/28 [02:11<00:00,  4.70s/it]


In [ ]:
df_logs = pd.DataFrame()
df_logs['login'] = logins
df_logs['subscribers'] = subscribers
df_logs['subscriptions'] = subscriptions
df_logs['Объем портфеля'] = cap
df_logs['Сделок за 30 дней'] = deals
df_logs['За год'] = percents

In [ ]:
df = df_posts.merge(df_logs, how='left', on='login')

In [ ]:
df['Объем портфеля'].unique()

array(['до 3000000 ₽', 'до 500000 ₽', 'до 50000 ₽', 'до 10000 ₽',
       'от 3000000 ₽', 'до 100000 ₽', 'до 1000000 ₽'], dtype=object)

In [ ]:
df.post.str.split(expand=True).stack().value_counts()

и             73
в             71
-             48
на            45
не            30
              ..
Roblox         1
рисков:        1
Среди          1
видеоигр:      1
задержкой.     1
Length: 1632, dtype: int64

In [ ]:
import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
import unicodedata
import collections

nltk.download("stopwords")
nltk.download('punkt')
russian_stopwords = stopwords.words("russian")
append_stopword = ['это', 'FXES', 'https', 'fxes', 'finex', 'финекс']
russian_stopwords.extend(append_stopword)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aleksandraromanenko/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/aleksandraromanenko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
mystem = Mystem() 

def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

In [ ]:
df['lemm_post'] = df.post.apply(preprocess_text)

In [ ]:
df.head()

,login,post,likes,time,subscribers,subscriptions,Объем портфеля,Сделок за 30 дней,За год,lemm_post
0,morcvec,"Посмотрел новый ETF от Финекса, $FXES, меня оч...",9,2021-07-15 15:07:00,3874,3,до 3000000 ₽,21,17.44,посмотреть новый etf очень смущать доля 7 5 4....
1,BonSai_108,Добавляю в мой долгосрочный портфель (ИИС- дру...,4,2021-07-15 15:07:00,2952,19,до 500000 ₽,184,19.57,добавлять долгосрочный портфель иис брокер нов...
2,vova.bless,$FXES 🕹 FinEx запустил новый фонд на видеоигры...,1,2021-07-15 15:07:00,41,19,до 500000 ₽,14,28.89,🕹 запускать новый фонд видеоигра киберспорт «...
3,Serg0787,$FXES как купить в долларах этот фонд?,1,2021-07-15 14:07:00,7,13,до 3000000 ₽,119,23.65,купить доллар фонд
4,ProInvestNews,#etf #новыйфонд #finex #разбор #new #игры #gam...,0,2021-07-15 14:07:00,107,27,до 500000 ₽,158,22.01,etf новыйфонд разбор new игра games 14 июль на...


In [8]:
!pip install dostoevsky

     |████████████████████████████████| 71kB 3.1MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3094794 sha256=8b47cec5ef2eebe9a2297692f41b7ff9e6e1bf546cb5eee0eccdf701cfae6560
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [9]:
!python -m dostoevsky download fasttext-social-network-model

In [10]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

In [11]:
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [12]:
messages = [
    'привет',
    'я люблю тебя!!',
    'малолетние дебилы'
]

results = model.predict(messages, k=2)

In [23]:
results

[{'skip': 0.0020607432816177607, 'speech': 1.0000100135803223},
 {'positive': 0.9886782765388489, 'skip': 0.005394937004894018},
 {'negative': 0.9525841474533081, 'neutral': 0.13661839067935944}]

In [24]:
for i, mes in enumerate(messages):
  tick = sorted(results[i])[0]
  print(mes, tick, results[i][tick])

привет skip 0.0020607432816177607
я люблю тебя!! positive 0.9886782765388489
малолетние дебилы negative 0.9525841474533081


In [31]:
sorted(results[2].items(), key=lambda x: x[1], reverse=True)[0]

('negative', 0.9525841474533081)

In [57]:
df= pd.read_csv('https://raw.githubusercontent.com/aromana33/FinEx/main/pulse_sentiment/df_upd.csv')

In [58]:
df.post = df.post.apply(str)

In [59]:
comments = df.post.tolist()

In [60]:
comments[0]

'посмотрел новый финекса очень смущают доли 7 этих sea limited unity bilibili целом вполне использовать вложений очень мелких сумм чисто ставка индустрию благодаря наличию крупной суммарной доли nvda tcehy nintendo atvi ea ttwo ubisoft capcom kingsoft уравновешивания множества перегретых ненадёжных скриншоте весь состав вчерашний день целом адекватно брать небольшую сумму увеличения экспозиции компаниях вроде тенсент нвидиа возможностью заработать дополнительную доходность случае долгосрочного успеха прочей рискованной мелочи лдв кроме прочего никуда исчезала фонды продолжаю покупать именно ради неё несмотря собранный портфель квалифицированных инвесторов igm qqq igv ihi psi отличие самых полезных фондов длинные облигации отрицательной реальной доходностью любимый управляющий предоставил российским инвесторам интересное прикуплю немного нвидиа уронит благодаря крупной доле коррекцией сплита ближайшем походе вниз рынка учётом успеха сделки arm компанию максимум 700 акцию оценить выходя 

In [61]:
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)
results = model.predict(comments, k=2)

In [90]:
from collections import defaultdict
funds = dict()
funds['FXES'] = dict()

neutral = []
positive = []
negative = []
speech = []
skip = []

for res in results:
  res = defaultdict(None, res)
  neutral.append(res.get('neutral'))
  positive.append(res.get('positive'))
  negative.append(res.get('negative'))
  speech.append(res.get('speech'))
  skip.append(res.get('skip'))
funds['FXES']['neutral'] = len([i for i in neutral if i])
funds['FXES']['positive'] = len([i for i in positive if i])
funds['FXES']['negative'] = len([i for i in negative if i])
funds['FXES']['speech'] = len([i for i in speech if i])
funds['FXES']['skip'] = len([i for i in skip if i])

In [91]:
funds

{'FXES': {'negative': 12,
  'neutral': 28,
  'positive': 9,
  'skip': 9,
  'speech': 2}}

In [81]:
from collections import defaultdict
f = defaultdict(None, results[0])

In [82]:
f

defaultdict(None,
            {'neutral': 0.5000100135803223, 'positive': 0.14805719256401062})

In [85]:
f.get('neutral')

0.5000100135803223

In [100]:
funds_n = ['TGRN', 'AMHC', 'TBIO']

In [93]:
tg_df = pd.read_csv('https://raw.githubusercontent.com/aromana33/FinEx/main/pulse_sentiment/TGRN_df_upd.csv')
am_df = pd.read_csv('https://raw.githubusercontent.com/aromana33/FinEx/main/pulse_sentiment/AMHC_df_upd.csv')
tb_df = pd.read_csv('https://raw.githubusercontent.com/aromana33/FinEx/main/pulse_sentiment/TBIO_df_upd.csv')

In [103]:
funds = dict()
for i, df in enumerate([tg_df, am_df, tb_df]):
  df.post = df.post.apply(str)
  comments = df.post
  tokenizer = RegexTokenizer()
  model = FastTextSocialNetworkModel(tokenizer=tokenizer)
  results = model.predict(comments, k=2)

  funds[funds_n[i]] = dict()

  neutral = []
  positive = []
  negative = []
  speech = []
  skip = []

  for res in results:
    res = defaultdict(None, res)
    neutral.append(res.get('neutral'))
    positive.append(res.get('positive'))
    negative.append(res.get('negative'))
    speech.append(res.get('speech'))
    skip.append(res.get('skip'))
  funds[funds_n[i]]['neutral'] = len([i for i in neutral if i])
  funds[funds_n[i]]['positive'] = len([i for i in positive if i])
  funds[funds_n[i]]['negative'] = len([i for i in negative if i])
  funds[funds_n[i]]['speech'] = len([i for i in speech if i])
  funds[funds_n[i]]['skip'] = len([i for i in skip if i])

In [105]:
for fund in funds_n:
  print(fund, sorted(funds[fund].items(), key=lambda x: x[1], reverse=True))

TGRN [('neutral', 26), ('negative', 17), ('positive', 9), ('skip', 6), ('speech', 2)]
AMHC [('neutral', 13), ('negative', 6), ('positive', 5), ('skip', 2), ('speech', 0)]
TBIO [('neutral', 29), ('negative', 17), ('skip', 9), ('positive', 4), ('speech', 1)]


In [109]:
for res in results:
  print(sorted(res.items(), key=lambda x: x[1], reverse=True))

[('skip', 0.22816647589206696), ('positive', 0.21207880973815918)]
[('neutral', 0.9362950325012207), ('skip', 0.24509501457214355)]
[('neutral', 0.8354935646057129), ('negative', 0.10088862478733063)]
[('neutral', 1.0000100135803223), ('skip', 0.0003669706347864121)]
[('neutral', 0.9219318628311157), ('positive', 0.3629792034626007)]
[('neutral', 0.4688006341457367), ('negative', 0.1732981950044632)]
[('neutral', 0.6001983880996704), ('negative', 0.22271016240119934)]
[('neutral', 0.7663036584854126), ('negative', 0.14034625887870789)]
[('neutral', 0.6370407938957214), ('skip', 0.23371636867523193)]
[('negative', 0.8634016513824463), ('neutral', 0.4532718360424042)]
[('neutral', 0.5775054097175598), ('negative', 0.16027602553367615)]
[('neutral', 0.607673168182373), ('positive', 0.13661839067935944)]
[('neutral', 0.4225146472454071), ('speech', 0.16886694729328156)]
[('neutral', 0.960371196269989), ('skip', 0.3276783227920532)]
[('neutral', 0.9539760947227478), ('skip', 0.1225332170724